## Initialization

In [1]:
# Import modules
import pandas as pd
import warnings

#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline

print("Packages installed")



Packages installed


In [2]:
def csv_to_df(filename):
    file_path = f"../DataSets/EncodedData/{filename}.csv"
    df = pd.read_csv(file_path, sep=",", encoding="UTF-8")
    return df

def df_to_csv(df, filename):
    df.to_csv(f"{filename}.csv", index=False)
    print(f"{filename}.csv er laget")
    return



## Read Clean data, drop some columns

In [25]:
df = csv_to_df("cleaned_data")

#,player_id,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,age_at_evaluation,domestic_competition_id,log_market_value_base10
import pandas as pd

# Sort DataFrame by year in descending order
df_sorted = df.sort_values(by='year', ascending=False)

# Drop duplicates based on 'player_id', keeping the first occurrence (which is the most recent)
df_no_duplicates = df_sorted.drop_duplicates(subset='player_id')

# Optionally, you can reset the index if you want
#df_no_duplicates = df_no_duplicates.reset_index(drop=True)

df_no_duplicates.head()

df_to_csv(df_no_duplicates, "cleaned_data_04_18")

df = df_no_duplicates
df.drop(["player_id", "country_of_birth", "date_of_birth", "date"], axis=1, inplace=True)
#df.dropna(inplace=True)

cleaned_data_04_18.csv er laget


In [26]:
df_ = df[df['name'].str.contains("Hasan K")]

print(df_)

                name country_of_citizenship  height_in_cm      sub_position  club_id     year  goals_for  goals_against  goals  assists  red_cards  yellow_cards  minutes_played  market_value_in_eur  age_at_evaluation domestic_competition_id  log_market_value_base10
61023    Hasan Kilic                 Turkey         183.0  Central Midfield   2944.0  2017.75        7.0           10.0    0.0      0.0        0.0           1.0           157.0             150000.0                 26                     TR1                 5.176091
91119  Hasan Kurucay                 Turkey         187.0       Centre-Back   5091.0  2017.25        4.0            4.0    0.0      0.0        0.0           0.0           180.0              25000.0                 20                     NaN                 4.397940
12133    Hasan Kabze                 Turkey         185.0    Centre-Forward   2381.0  2016.25       15.0           23.0    1.0      0.0        0.0           0.0           785.0             200000.0     

## TargetEncode: Country, Position, Club, League

In [27]:
from category_encoders.target_encoder import TargetEncoder

# Initialize TargetEncoder
encoder = TargetEncoder()

# Specify columns to target encode
columns_to_encode = ['country_of_citizenship', 'sub_position', 'club_id', 'domestic_competition_id']

df['club_id'] = df['club_id'].astype(str)

# Fit and transform the DataFrame with target encoding
df_encoded = encoder.fit_transform(df[columns_to_encode], df['market_value_in_eur'])

# Merge the encoded DataFrame with the original DataFrame
df = pd.concat([df, df_encoded.add_suffix('_encoded')], axis=1)
df = df.drop(["country_of_citizenship", "sub_position", "domestic_competition_id", "club_id"], axis=1)
# Display the resulting DataFrame
print(df.head())
df_to_csv(df, "encoded_data_04_18")


                       name  height_in_cm     year  goals_for  goals_against  goals  assists  red_cards  yellow_cards  minutes_played  market_value_in_eur  age_at_evaluation  log_market_value_base10  country_of_citizenship_encoded  sub_position_encoded  club_id_encoded  domestic_competition_id_encoded
120798  Franciszek Franczak         173.0  2023.75        2.0            6.0    0.0      0.0        0.0           1.0           233.0              75000.0                 16                 4.875061                    2.553086e+06          1.093889e+06     3.319356e+05                     5.924867e+05
111136       Denys Norenkov         181.0  2023.75       14.0           17.0    1.0      1.0        0.0           4.0          1210.0             400000.0                 27                 5.602060                    7.122837e+05          1.882187e+06     1.342678e+06                     5.604821e+05
111140         Henok Teklab         174.0  2023.75       13.0           12.0    0.0      1.

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame
columns_to_normalize = ["goals", "assists", "minutes_played", "height_in_cm", "year", "goals_for", "goals_against", "red_cards", "yellow_cards", "age_at_evaluation"]

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
df.head()

df_to_csv(df, "encoded_data_04_16")

encoded_data_04_16.csv er laget


In [27]:
df = csv_to_df("encoded_data_04_16")

top_5_competition_ids = df["domestic_competition_id_encoded"].value_counts().nlargest(5).index
df = df[df["domestic_competition_id_encoded"].isin(top_5_competition_ids)]

df_to_csv(df, "encoded_data_04_16_top5.csv")

(52500, 19)